# Introduction to Postgres

## RDBMS

관계형 데이터베이스 관리 시스템(Rrelational Database Management System; RDBMS)는 다수의 사용자들이 데이터베이스 내의 데이터를 접근할 수 있도록 해주는 소프트웨어 도구의 집합이다. DBMS는 사용자 또는 다른 프로그램의 요구를 처리하고 적절히 응답하여 데이터를 사용할 수 있도록 해준다. 

PosgreSQL은 확장 가능성 및 표준 준수를 강조하는 관계형 데이터베이스 관리 시스템의 하나로 데이터베이스 서버로서 주요 기능은 데이터를 안전하게 저장하고 다른 응용 소스 소프트웨어로부터 요청에 응답할 때 데이터를 반환하는 작업을 수행한다. 소규모 단일 머신 애플리케이션에서부터 수많은 동시 접속자가 있는 대형 인터넷 애플리케이션에 이르기까지 여러 부하를 관리할 수 있다. 

## Postgres in Python Script

psycopg2는 파이썬 스크립트에서 Postgres를 사용하기 위한 패키지이다. sqlite3와 동일하게 conn 인스턴스와 cursor 인스턴스를 생성해 쿼리를 작성한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

query = """SELECT * FROM users"""
cursor.execute(query)
user = cursor.fetchall() 

conn.close()
```

외부 데이터를 Postgres 데이터베이스 내부 테이블에 작성하기 위해서 INSERT INTO 절과 지시자(%s)를 사용하면 수월한 작업을 할 수 있다. cursor.execute() 메소드는 두번째 argument로 지시자로 처리하기 위한 데이터 셋을 입력 받는다. 

```Python
import psycopg2
import csv
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

with open("user_accounts.csv", "r") as file :
    next(file)
    reader = csv.reader(file)
    for row in reader : 
        cursor.execute("INSERT INTO users VALUES (%s, %s, %s, %s);", (row[0], row[1], row[2], row[3]))
        
conn.commit()
conn.close() 
                      
```

## SQL transcation

데이터베이스 트랜잭션(Database Trasaction)은 데이터베이스 관리 시스템 또는 유사한 시스템에서 상호 작용의 단위이다. 여기서 유사한 시스템이란 트랜잭션의 성공과 실패가 분명하고 상호 독립적이며, 일관되고 믿을 수 있는 시스템을 의미한다. 

Postgres는 서버와 연결된 다수의 사용자가 데이터베이스를 동시에 변경하거나 다루기 때문에 문제가 발생할 수 있다. 트랜잭션은 따로따로 실행되거나 실행되지 못했을 때 문제가 발생할 수 있는 모든 쿼리를 트랜잭션 블록에 넣어 한번에 실행시킨다. 따라서 하나의 트랜잭션이 실패하면 모든 트랜잭션이 실패하게 되고 데이터베이스 정보가 업데이트 되지 않는다.

Postgres 내부에는 connect.commit() 메소드를 통해서 트랜잭션의 쿼리를 실행한다. conn 인스턴스가 생성되고 close() 메소드를 통해 닫힐 떄까지 실행 되는 모든 쿼리가 트랜잭션 블록이 된다. 

커밋한 사항을 적용하고 싶지 않으면 connect.rollback() 메소드를 통해 트랜잭션을 제거할 수 있다. 

```Python
import psycopg2 
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query_string = """
    CREATE TABLE users(
        id integer PRIMARY KEY,
        email text,
        name text,
        addres text 
    );
"""

cursor.execute(query_string)
conn.commit()
conn.close()
```

# Creating Tables 

## Check Datatype 

적절하지 않은 데이터 타입 스키마는 쿼리문을 작성할 때 오류를 발생시킨다. psycopg2 패키지의 cursor.description 속성은 데이터베이스 테이블의 데이터 타입을 출력한다. curosr.description 속성은 name과 type_code로 이루어진 Column 객체로 구성되어 있다. name은 열의 이름을, type_code는 데이터 타입에 상응하는 정수 값을 가지고 있다. pg_catalog.pg_type 테이블은 type_code(oid)와 데이터 타입 이름(typename)으로 구성되어 있다.


| Name | Type | Description |
|:---:|:---:|:---:|
| oid | oid | Row identifier | 
| typename | name | Data type name | 


```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "SELECT * FROM table LIMIT 0;" 
curosr.execute(query)
print(cursor.description)

# The type_code will be in 'nn'

query = "SELECT typename FROM pg_catalog.pg_type WHERE oid=nn;"
cursor.execute(query)
type_of_nn = cursor.fetchone()[0]
print(f"The typename of type code : {type_of_nn}")
```

## Optimizing Numerical variable

데이터베이스 테이블은 데이터를 저장하는 공간으로써 저장 공간이 필요하다. 만약 불필요하게 넓은 크기를 가지고 있는 데이터 타입을 가지고 있다면 공간 낭비와 속도 낭비를 초래한다. 따라서 데이터 타입을 최적화 하는 작업은 반드시 필요하다. 모든 데이터를 오류 없이 출력 가능한 데이터 타입을 선정한 후 ALTER 절을 사용해 데이터 타입을 변경한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "ALTER TABLE users ALTER COLUMN id TYPE int4;"
cursor.execute(query)

conn.commit()
conn.close()
```

## Optimizing Text variable

텍스트 데이터의 경우 테이블 내부에 어떤 길이의 문자열도 저장 가능하다. 이런 유연성은 데이터를 저장하는데 필요한 공간의 낭비를 불러온다. 따라서 각 열의 고유값에 대해 가장 긴 길이의 문자열을 기준으로 데이터의 크기를 제한하는 작업이 필요하다. 

```Python
import csv 
with open('users.csv', 'r') as file : 
    next(file)
    reader = csv.reader(file)
    unique_words = set()
    for row in reader : 
        check_col = row[n]
        unique_words.add(score_phrase)
        
max_len = 0
for unique_word in unique_words : 
    if len(unique_word) >= max_len : 
        max_len = len(unique_word) 
print(max_len)

import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = "ALTER TABLE user ALTER COLUMN email TYPE varcahr(11);"
cursor.execute(query)

conn.commit()
conn.close() 
```

## Optimizing using enumerated datatype 

모든 고유값을 확인해서 가장 큰 길이의 문자열을 확인하는 작업보다 열거형 데이터 타입을 사용하게 되면 좀 더 효율적인 작업이 간으하다. 열거형 타입은 명명된 값의 집합을 이루는 자료형으로, 해당 언어의 상수 역할을 하는 식별자이다. 열거형 자료는 각각의 색인에 4바이트가 할당되어 pg_enum 테이블에 저장된다. 한번 저장된 열거형 자료는 Postgres 엔진이 값을 바로 확인해서 저장할 수 있도록 한다.

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = """
CREATE TYPE enum_name AS ENUM(
    'val1', 'val2', 'val3', ... 
); 

ALTER TABLE table_name 
ALTER COLUMN column TYPE enum_name
USING CAST(col_name AS enum_name);
"""
cursor.excute(query)

conn.commit()
conn.close() 
```

## Create new variable and insert 

```Python
import datetime
import psycopg2
import csv
conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

query1 = "ALTER TABLE ign_reviews ADD COLUMN release_date date;"
query2 = """
ALTER TABLE ign_reviews
DROP COLUMN release_year,
DROP COLUMN release_month,
DROP COLUMN release_day;
"""
cur.execute(query1)
cur.execute(query2)

with open('ign.csv', 'r') as file :
    next(file)
    reader = csv.reader(file)
    for row in reader : 
        year = int(row[8])
        month = int(row[9])
        day = int(row[10])
        date = datetime.date(year, month, day)
        row = row[:-3]
        row.append(date)
        query = "INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
        cur.execute(query, row)

conn.commit()
conn.close()
```

# Prepared Statement and SQL injection

## SQL injection

SQL injection은 응용 프로그램 보안 상의 허점을 의도적으로 이용해, 악의적인 SQL 문을 실행되게 함으로써 데이터베이스를 비정상적으로 조작하는 코드 인젝션 공격 방법이다. 단순히 값을 받아서 수행하는 쿼리문에 "admin OR 1=1;--"를 입력하면 쿼리문은 모든 행을 출력하게 된다. 

```Python
query = "SELECT * FROM users WHERE username = '" + name + "';"

# if name = "admin OR 1=1;--"
query = "SELECT * FROM users WHERE username = admin OR 1=1;"
```

## %s Placeholder

Postgres의 cursor.execute() 메소드는 자리 표시자(placholder)를 입력받아 데이터베이스를 조작할 수 있다. 함수 내부에 입력 가능한 자리 표시자는 두가지가 있다.

- %s : 순서에 따라 값을 대체하는 자리 표시자
- %(key)s : key값에 따라 값을 대체하는 자리 표시자 

%(key)s 방법의 경우 튜플, 리스트 대신에 딕셔너리 형태로 존재하는 key : value를 입력해야 한다.

```Python
import psycpog2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

query = """INSERT INTO users VALUES (%s, %s, %s, ..., %s);"""
cursor.execute(qeury, ('val1', 'val2', ... 'valn'))

conn.commit()
conn.close()
```

```Python
import psycpog2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

row_dict = {'key1' : 'val1' : 'key2' : 'val2', ... , 'keyn' : 'valn'}
query = """INSERT INTO users VALUES (%(key1)s, %(key2)s, ..., %(keyn)s);"""
cursor.execute(qeury, row_dict)

conn.commit()
conn.close()
```

## Prepared Statement 

Prepared Statement는 추가적으로 뒤따라오는 SQL 문을 무시하고 지정되어 있는 형태의 쿼리를 수행할 수 있도록 만든다. 즉 Prepared Statement는 SQL injection을 방어하기 위해 미리 지정된 함수와 같은 개념으로 사용된다. 지정된 Prepared Statement는 EXCUTE 절을 사용해 실행한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

cursor.execute("""
    PREPARE pre_name(int4, text, text, ... ) AS 
        INSERT INTO users VALUES ($1, $2, $3, ..., $n)
""")

cursor.execute("EXECUTE pre_name(%s, %s, ..., %s)", ('val1', 'val2', ..., 'valn'))

conn.commit()
conn.close()
```

# Loading and Extracting Data with Tables 

## What is mogrifying 

Python script에서 Postgres 데이터베이스를 다룰때 외부의 값을 사용하려면 placeholder를 사용해 튜플을 추가적으로 입력해야한다. cursor.execute() 메소드에 전달된 값은 자동으로 Postgres 내부에서 읽을 수 있는 값으로 변환되어 전송되어야 한다. curor.mogrify() 메소드는 Postgres 엔진이 읽을 수 있는 바이트 객체로 데이터를 변환해준다. 

```Python
mogrified_data = cur.mogrify("(%s, ..., %s)", tuples)

# decode mogrified_data
mogrified_string = mogrified_data.decode(conn.encoding)
```

## Loading data with cursor.mogrify()

외부 데이터를 Postgres 데이터베이스 내부에 입력하기 위해서는 지시자를 통해 튜플 값을 cursor.execute()에 넣고, cursor.execute() 내부에서 Postgres 엔진이 읽을 수 있는 값으로 mogrify 한 뒤에 값을 추출한다. cursor.mogrify()를 진행한뒤 일반적인 string 형식으로 쿼리문을 작성하게 되면 한번에 많은 양의 반복 쿼리를 빠르게 수행할 수 있다. 

cursor.mogrify() 메소드를 사용하고 디코딩 된 데이터들은 placeholder를 통해 정의되어 있기때문에 SQL injection 문제에 대해서도 어느정도 자유롭다. 

```Python
import csv
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

with open("ign.csv", mode = 'r') as file : 
    next(file)
    reader = csv.reader(file) 
    rows = [row for row in reader]
    
    mogrified_rows = [cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row) for row in rows]
    decoded_rows = [row.decode(conn.encoding) for row in mogrified_rows]
    insert_string = ",".join(decoded_rows)
    cur.execute("INSERT INTO ign_reviews VALUES " + insert_string + ";")
    conn.commit()
    conn.close()
```

## Loading data with cursor.copy_from() 

cursor.mogrify()는 지시자를 사용하는 방식보다는 빠르긴 하지만 결국 각 행을 하나씩 읽어들이고 디코딩 한 이후 join 연산을 해서 쿼리문을 수행해야 한다. cursor.copy_from() 메소드는 모든 행을 복사하는 경우에 대해서 cursor.mogrify() 메소드 보다 빠르게 전체 데이터를 옮길 수 있다. 


```Python
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

with open("ign.csv", mode = 'r') as file : 
    next(file) 
    cur.copy_from(file, "ign_reviews", sep = ',')
```

하지만 cur.copy_from() 메소드는 csv 파일을 읽을 때, 각 데이터가 ','가 있을 경우 분할된 열로 인식하기 때문에 오류가 발생할 수 있다. cur.copy_export() 메소드는 세부적인 옵션을 작성함으로써 데이터 로딩 작업을 수월하게 할 수 있다. 데이터를 테이블로 옮기기 위해서는 COPY ~ FROM STDIN 쿼리문을 사용하면 된다. 

```Python
import csv
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

with open('ign.csv', mode = 'r') as file :
    cursor.copy_expert("COPY ign_reviews FROM STDIN CSV WITH HEADER", file)
conn.commit()
conn.close()
```

## Fastest way to loading data

파이썬 스크립트에서 Postgres DB에 데이터를 업로딩한다고 했을때 가능한 방법은 3가지가 있다.

1. 지시자를 활용하여 각각의 행을 개행으로 입력하는 방법
2. cursor.mogrify()를 이용해 모든 행을 한번에 업로드 하는 방법
3. cursor.copy_expert()를 이용해 데이터 전체를 한번에 테이블로 업로드 하는 방법

timeit 패키지의 timetit() 함수를 활용해서 각각의 방법의 시간을 측정한 결과 3, 2, 1 순서로 속도가 빨랐다. 

```Python
import csv
import psycopg2

conn = psycopg2.connect("dbname=dq user=dq")
cur = conn.cursor()

# Multiple single insert statements
def multiple_inserts():
    with open("ign.csv", "r") as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            cur.execute("INSERT INTO ign_reviews VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);", row)
    conn.rollback()
        
# Multiple mogrify insert
def mogrified_insert():
    with open("ign.csv", "r") as f:
        next(f)
        reader = csv.reader(f)
        mogrified = [ 
            cur.mogrify("(%s, %s, %s, %s, %s, %s, %s, %s, %s)", row).decode(conn.encoding)
            for row in reader
        ] 
        mogrified_values = ",".join(mogrified) 
        cur.execute("INSERT INTO ign_reviews VALUES " + mogrified_values + ";")
    conn.rollback()
    
# Copy expert method
def copy_expert():
    with open("ign.csv", "r") as f:
        cur.copy_expert("COPY ign_reviews FROM STDIN WITH CSV HEADER;", f)
    conn.rollback()

# Add your code under
import timeit

time_multiple_inserts = timeit.timeit(multiple_inserts, number = 1)
time_mogrified_insert = timeit.timeit(mogrified_insert, number = 1)
time_copy_expert = timeit.timeit(copy_expert, number = 1)

print(f"Consumed times using multiple insert : {time_multiple_inserts}")
print(f"Consumed times using mogrified insert : {time_mogrified_insert}")
print(f"Consumed times using copy expert : {time_copy_expert}")
```

## Extracting data

cursor.copy_expert() 메소드는 데이터를 업로딩 하는것 뿐만 아니라 테이블의 데이터를 파일로 추출할 수 있다. 이때 FROM을 TO, STDIN을 STDOUT으로 변경하고 파일을 오픈할 때 'w' 모드로 작성해야 한다. 

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

with open("ign_copy.csv", "w") as file : 
    cursor.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER;", file)
    conn.close()
```

## Trasnferring data between tables 

두 테이블 간 데이터를 전송하는 방법은 두가지가 있다. 첫번째는 기존의 테이블의 데이터를 보존할 temporary file을 생성하고 cursor.copy_expert("COPY ... TO STDOUT ...")으로 데이터를 외부 저장소에 저장한 뒤 cursor.copy_expert("COPY ... FROM STDIN ...") 메소드로 테이블에 옵로딩 하는 방법이 있다. 하지만 해당 방법은 데이터의 행이 수백만개가 넘을경우 시간, 공간상 효율 문제가 발생한다. 두번째 방법은 SQL 내부에서 테이블 간 데이터를 전송하는 방법이다. INSERT INTO 쿼리문에서 VALUES를 서브쿼리로 작성해서 전송하고자 하는 테이블의 데이터를 전송할 수 있다. 

```Python
import psycopg2
# the query for you to create the empty table copy
create_string = """
CREATE TABLE ign_reviews_copy (
    id bigint PRIMARY KEY,
    score_phrase evaluation_enum,
    title varchar(200),
    url varchar(200),
    platform platform_enum,
    score decimal(3, 1),
    genre genre_enum,
    editors_choice boolean,
    release_date date
);
"""

conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor() 

with open('temp.csv', 'w') as file :
    cursor.copy_expert("COPY ign_reviews TO STDOUT WITH CSV HEADER;", file)
cursor.execute(create_string)
with open('temp.csv', 'r') as file :
    cursor.copy_expert("COPY ign_reviews_copy FROM STDIN WITH CSV HEADER;", file)
conn.commit()
conn.close()
```

```Python
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
cursor = conn.cursor()

cursor.execute("""CREATE TABLE ign_restricted (
    id bigint PRIMARY KEY,
    title varchar(200),
    release_date date
);""")
cursor.execute("""
    INSERT INTO ign_restricted (id, title, release_date) 
    	SELECT id, title, release_date FROM ign_reviews;
""")

conn.commit()
```

# Users and Database Management 

# Exploring Postgres Internals

# Debugging Postgres Queries 

# Using an Index

# Advanced Index

# Vacumming Postgres Database 